## 9.2 EM算法在高斯混合模型学习中的应用

参考《白板推导系列》  

### 9.2.1 GMM

$x$ 表示观测随机变量  

$z$ 表示隐变量  


为了解决<font color=red>高斯模型的单峰性问题</font>，我么引入多个高斯模型的加权平均来模拟多峰数据:  

$$
p(x)=\sum_{k=1}^{K} \alpha_{k} \mathcal{N}\left(\mu_{k}, \Sigma_{k}\right) \tag{1}
$$

引入隐变量$z$，这个变量表示对应的样本$x$ 属于哪一个高斯分布，这个变量是一个离散的随机变量:  

$$
p(z=i)=p_{i},\  \sum_{i=1}^{k} p(z=i)=1 \tag{2}
$$

作为一个生成模型，GMM通过隐变量$z$的分布来生成样本，用有向图表示为  

<img src='9.3_GMM.png' style='zoom:50%'/>

其中，节点$z$就是上面的概率，$x$就是生成的高斯分布。于是对$p(x)$:  

$$
p(x) = \sum_{z}p(x,z) = \sum_{k=1}^{K}p(x,z=k) = \sum_{k=1}^{K}p(z=k)p(x|z=k) \tag{3}
$$

因此  

$$
\color{red}{p(x) = \sum_{k=1}^{K}p_{k}\mathcal{N}(x|u_{k},\Sigma_{k})} \tag{4}
$$

### 9.2.2 MLE

样本为$X=\left(x_{1}, x_{2}, \cdots, x_{N}\right)$（$X,Z$）为完全参数，参数为$\theta=\left\{p_{1}, p_{2}, \cdots, p_{K}, \mu_{1}, \mu_{2}, \cdots, \mu_{K}, \Sigma_{1}, \Sigma_{2}, \cdots, \Sigma_{K}\right\}$，通过极大似然估计得到$\theta$的值:  

$$
\begin{eqnarray}
\theta_{MLE} &=& \arg \max_{\theta} \log p(X) = \arg \max_{\theta} \sum_{i=1}^{N}\log p(x_{i}) \\
&=& \arg \max_{\theta} \sum_{i=1}^{N}\log \sum_{k=1}^{K} p_{k}\mathcal{N}(x_{i}|u_{k},\Sigma_{k}) \tag{5}
\end{eqnarray}
$$

由于$\sum$的存在，很难直接求导得到解析解，因此使用EM算法。

### 9.2.3 EM求解GMM

EM算法的基本表示为  

$$
\theta^{(t+1)} = \arg \max_{\theta} \mathbb{E}_{z|x,\theta^{(t)}} \left[p \left(x,z|\theta \right)\right]
$$

对整个数据集来说，$Q(\theta, \theta^{(t)})$函数为:  

$$
\begin{eqnarray}
Q\left(\theta, \theta^{(t)}\right) &=& \sum_{z}\left[\log \prod_{i=1}^{N} p\left(x_{i}, z_{i} \mid \theta\right)\right] \prod_{i=1}^{N} p\left(z_{i} \mid x_{i}, \theta^{(t)}\right) \\
&=& \sum_{z}\left[\sum_{i=1}^{N} \log p\left(x_{i}, z_{i} \mid \theta\right)\right] \prod_{i=1}^{N} p\left(z_{i} \mid x_{i}, \theta^{(t)}\right) \tag{6}
\end{eqnarray}
$$

对于中间求和项，展开，讨论$i=1$时的情况   

$$
\begin{eqnarray}
\sum_{z} \log p\left(x_{1}, z_{1} \mid \theta\right) \prod_{i=1}^{N} p\left(z_{i} \mid x_{i}, \theta^{(t)}\right) &=& \sum_{z} \log p\left(x_{1}, z_{1} \mid \theta\right) p\left(z_{1} \mid x_{1}, \theta^{(t)}\right) \prod_{i=2}^{N} p\left(z_{i} \mid x_{i}, \theta^{(t)}\right)  \\
&=& \sum_{z_{2},\cdots, z_{K}}\sum_{z_{1}} \log p\left(x_{1}, z_{1} \mid \theta\right) p\left(z_{1} \mid x_{1}, \theta^{(t)}\right) \prod_{i=2}^{N} p\left(z_{i} \mid x_{i}, \theta^{(t)}\right) \\
&=& \sum_{z_{1}} \log p\left(x_{1}, z_{1} \mid \theta\right) p\left(z_{1} \mid x_{1}, \theta^{(t)}\right) \sum_{z_{2}, \cdots, z_{K}} \prod_{i=2}^{N} p\left(z_{i} \mid x_{i}, \theta^{(t)}\right) \\
&=& \sum_{z_{1}} \log p\left(x_{1}, z_{1} \mid \theta\right) p\left(z_{1} \mid x_{1}, \theta^{(t)}\right) \tag{7}
\end{eqnarray}
$$

当$i=2,\cdots,N$时，与上式类似，因此，$Q(\theta, \theta^{(t)})$函数可写成 

$$
Q\left(\theta, \theta^{t}\right)=\sum_{i=1}^{N} \sum_{z_{i}} \log p\left(x_{i}, z_{i} \mid \theta\right) p\left(z_{i} \mid x_{i}, \theta^{(t)}\right) \tag{8}
$$

其中  

$$
\begin{eqnarray}
p(x, z | \theta) &=& p(z | \theta) p(x | z, \theta) \\
&=& p_{z} \mathcal{N}\left(x | \mu_{z}, \Sigma_{z}\right) \tag{9}
\end{eqnarray}
$$

对$p(z|x,\theta^{(t)})$

$$
p\left(z | x, \theta^{(t)}\right)=\frac{p\left(x, z | \theta^{(t)}\right)}{p\left(x | \theta^{(t)}\right)}=\frac{p_{z}^{(t)} \mathcal{N}\left(x | \mu_{z}^{(t)}, \Sigma_{z}^{(t)}\right)}{\sum_{k} p_{k}^{(t)} \mathcal{N}\left(x | \mu_{k}^{(t)}, \Sigma_{k}^{(t)}\right)} \tag{10}
$$

将（9）和（10）带入（8）得$Q$函数  

$$
\begin{eqnarray}
Q &=& \sum_{i=1}^{N} \sum_{z_{i}} \log p_{z_{i}} \mathcal{N}\left(x_{i} | \mu_{z_{i}}, \Sigma_{z_{i}}\right) \frac{p_{z_{i}}^{(t)} \mathcal{N}\left(x_{i} | \mu_{z_{i}}^{(t)}, \Sigma_{z_{i}}^{(t)}\right)}{\sum_{k} p_{k}^{(t)} \mathcal{N}\left(x_{i} | \mu_{k}^{(t)}, \Sigma_{k}^{(t)}\right)}\\
&=& \sum_{k=1}^{K} \sum_{i=1}^{N}\left[\log p_{k}+\log \mathcal{N}\left(x_{i} | \mu_{k}, \Sigma_{k}\right)\right] p\left(z_{i}=k | x_{i}, \theta^{(t)}\right) \tag{11}
\end{eqnarray}
$$

#### 求$p_{k}^{(t+1)}$

$$
\begin{eqnarray}
p_{k}^{(t+1)} &=& \arg \max_{p_{k}} \sum_{k=1}^{K} \sum_{i=1}^{N}\left[\log p_{k}+\log \mathcal{N}\left(x_{i} | \mu_{k}, \Sigma_{k}\right)\right] p\left(z_{i}=k | x_{i}, \theta^{(t)}\right) \\
&\text{s.t.}& \sum_{k=1}^{K}p_{k}=1 \tag{13}
\end{eqnarray}
$$

等价于  

$$
\begin{eqnarray}
p_{k}^{(t+1)} &=& \arg \max_{p_{k}} \sum_{k=1}^{K} \sum_{i=1}^{N}\log p_{k} \ p\left(z_{i}=k | x_{i}, \theta^{(t)}\right) \\
&\text{s.t.}& \sum_{k=1}^{K}p_{k}=1 \tag{14}
\end{eqnarray}
$$

引入Lagrange乘子，得Lagrange函数  

$$
L\left(p_{k}, \lambda\right)=\sum_{k=1}^{K} \sum_{i=1}^{N} \log p_{k} p\left(z_{i}=k | x_{i}, \theta^{(t)}\right)-\lambda\left(1-\sum_{k=1}^{K} p_{k}\right)
$$

对$p_{k}$求偏导，并令导数等于0，有  

$$
\begin{eqnarray}
\frac{\partial}{\partial p_{k}} L &=& \sum_{i=1}^{N} \frac{1}{p_{k}} p\left(z_{i}=k | x_{i}, \theta^{(t)}\right)+\lambda=0 \\
&\Rightarrow& \sum_{i=1}^{N} p\left(z_{i}=k | x_{i}, \theta^{(t)}\right)+\lambda p_{k}=0 \\
&\Rightarrow& \sum_{i=1}^{N}\sum_{k} p\left(z_{i}=k | x_{i}, \theta^{(t)}\right)+\lambda \sum_{k}p_{k}=0 \\
&\Rightarrow& \lambda = -N \tag{15}
\end{eqnarray}
$$

于是  

$$p_{k}^{(t+1)}=\frac{1}{N} \sum_{i=1}^{N} p\left(z_{i}=k | x_{i}, \theta^{(t)}\right)$$

#### 求$u_{k},\Sigma_{k}$  

只需将式（11）分别对$u_{k},\Sigma_{k}$求偏导并令其为0，即可得到。

$$
u_{k}^{(t+1)}=\frac{\sum_{i=1}^{N} p\left(z_{i}=k | x_{i}, \theta^{(t)}\right) x_{i}}{\sum_{i=1}^{N} p\left(z_{i}=k | x_{i}, \theta^{(t)}\right)}, \quad k=1,2, \cdots, K
$$

$$
\hat{\Sigma}_{k}^{2}=\frac{\sum_{j=1}^{N} \hat{\gamma}_{j k}\left(y_{j}-\mu_{k}\right)^{2}}{\sum_{j=1}^{N} \hat{\gamma}_{j k}}, \quad k=1,2, \cdots, K
$$